In [1]:
import numpy as np
from utils import *
import pandas as pd

In [2]:
data = pd.read_table("data/bluebird_crowd.txt",header=None)

In [14]:
truth = pd.read_table("data/bluebird_truth.txt", header=None)


In [3]:
def transform_data(data):
    X = np.array(data)
    rows, _ = X.shape
    n,m,k = np.max(np.array(data), axis=0)
    labels = np.zeros((m,n),dtype=np.int64)
    for r in range(rows):
        labels[X[r,1]-1,X[r,0]-1] = X[r,2]-1
    return labels


In [4]:
n,m,k = np.max(np.array(data),axis=0)
print(n,m,k)

108 39 2


In [5]:
labels = transform_data(data)


In [22]:
C_estimated = get_confusion_matrix(k,labels)

In [17]:
data[data[1]==3][2]

2       2
41      2
80      2
119     2
158     2
       ..
4019    2
4058    2
4097    1
4136    1
4175    1
Name: 2, Length: 108, dtype: int64

In [15]:
from sklearn.metrics import confusion_matrix


In [36]:
C = confusion_matrix(data[data[1] == 4][2],truth[1])

In [37]:
C / np.sum(C,axis=0)[np.newaxis,:]

array([[0.15  , 0.1875],
       [0.85  , 0.8125]])

In [35]:
C_estimated[3]

array([[0.20432029, 0.11541396],
       [0.79567971, 0.88458604]])